# Import des données DAS dans base de données SQLite

In [24]:
import pandas as pd
import os
import sqlite3 as sql

## Création de la base de données et des tables

* À partir du dossier contenant les tables de la base de données originale exportée en CSV, on le lit et importe dans une nouvelle base de données SQLite.
* On recrée ensuite manuellement les clés primaires et les liens de clé étrangère


In [3]:
### Connection à la base de données SQLite
# Si elle n'existe pas elle est créée à la connection
db_address = '/home/francesco/shared_files/python_notebooks/SD_for_HSS/Cycle-Data/data/maritime_history.db'
cnn = sql.connect(db_address)

### Importer les données

In [25]:
### Chemin vers le dossier
file_path = '[chemin]/das'

In [46]:
### Lister les fichiers dans le dossier
dir_list = os.listdir(file_path)
print(dir_list)

['das_master.csv', 'das_shipNameVariant.csv', 'das_yard.csv', 'das_ship.csv', 'das_ship_test_noms_champs.csv', 'das_shipType.csv', 'das_place.csv', 'das_onboard.csv', 'das_voyage.csv', 'das_chamber.csv']


In [17]:
### Explorer le premier fichier
csv_file = dir_list[0]
f = pd.read_csv(file_path + '/' + csv_file, sep=';')
f.head()

,voyMasterID,voyMasterLastName,voyMasterFirstName,voyMasterFamilyNamePrefix
0,das_mas0001,NaN,Evert,NaN
1,das_mas0002,Aalders,Aldert,NaN
2,das_mas0003,Aams,Lukas,NaN
3,das_mas0004,Aarnoutsz.,Abraham,NaN
4,das_mas0005,Aarson,Pieter,van


In [ ]:
### Boucle qui importe tous les fichiers CSV
# et créé une table par fichier dans la base de données

for f in dir_list:
    # Lire le fichier
    fdf = pd.read_csv(file_path + '/' + f, sep=';')
    # Préparer le nom de la table
    tb_name = f.replace('.csv', '')
    ### La base de données existe déjà, ne pas rééxécuter 
    # fdf.to_sql(tb_name, cnn)
    q = 'update ' + tb_name + ' set "index" = "index" + 1'
    ### Déjà exécuté, ne pas rééxécuter 
    # cnn.execute(q)

### Inspecter le résultat de l'importation

In [50]:
q = """SELECT 
    name
FROM 
    sqlite_schema
WHERE 
    type ='table' 
    AND name NOT LIKE 'sqlite_%';
    """
c = cnn.cursor()
c.execute(q)
rc = c.fetchall()

In [51]:
len(rc), print(rc)

[('das_master',), ('das_ship_test_noms_champs',), ('das_onboard',), ('das_chamber',), ('das_ship',), ('das_shipType',), ('das_shipNameVariant',), ('das_voyage',), ('das_place',), ('das_yard',)]


(10, None)

In [52]:
q = 'SELECT * FROM ' + rc[0][0] 
ct = cnn.cursor()
ct.execute(q)
rct = ct.fetchall()

In [55]:
len(rct), rct[:5]

(3073,
 [(1, 'das_mas0001', None, 'Evert', None),
  (2, 'das_mas0002', 'Aalders', 'Aldert', None),
  (3, 'das_mas0003', 'Aams', 'Lukas', None),
  (4, 'das_mas0004', 'Aarnoutsz.', 'Abraham', None),
  (5, 'das_mas0005', 'Aarson', 'Pieter', 'van')])

In [56]:
# fermer la connexion
cnn.close()